# Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import random as rd
import os
from tqdm import tqdm
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar


plt.style.use(['science','ieee'])

os.chdir('/home/mathis/Stage M1/')

2022-06-10 16:52:15.448693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 16:52:15.448722: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Model

In [2]:
fnc_activation = 'tanh'
fnc_activation_output = 'sigmoid'
inputs = tf.keras.Input(shape=(5,), dtype='float32')
x = keras.layers.Dense(100, trainable=True, activation=fnc_activation)(inputs)
x = tf.keras.layers.Dropout(1e-2, input_shape=(100,))(x)
x = keras.layers.Dense(100, trainable=True, activation=fnc_activation)(x)
outputs = keras.layers.Dense(19, activation=fnc_activation_output)(x)
model = tf.keras.Model(inputs,outputs)

lr = 1e-3

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

model.summary()



loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction='sum_over_batch_size')
#False pour une probabilité ie [0;1]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense (Dense)               (None, 100)               600       
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 19)                1919      
                                                                 
Total params: 12,619
Trainable params: 12,619
Non-trainable params: 0
_________________________________________________________________


2022-06-10 16:52:18.004524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-10 16:52:18.004551: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-10 16:52:18.004571: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MPC): /proc/driver/nvidia/version does not exist
2022-06-10 16:52:18.004775: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Initial training

In [3]:
path = 'Advanced NN/AL/AL_run/'
folder_r = '2022-06-09-06:25:27/'
file = 'K_pool.csv'

import csv

with open(path+folder_r+file) as f:
    f.readline()
    data = csv.reader(f, delimiter='\t')
    data = list(data)
    

input_pred = []
good_data = []
bad_data = []

for i in data:
    input_pred.append([rd.uniform(0,1) for j in range(5)])
    if float(i[-1]) == 0:
        bad_data.append([float(i[j]) for j in range(19)])
    if float(i[-1]) == 1:
        good_data.append([float(i[j]) for j in range(19)])
    

In [4]:
def Oracle(logits, PMSSM_range, ratio, succes_path):
    
    PMSSM_AL = np.add(np.multiply(logits,PMSSM_range[2]),PMSSM_range[0])
    write_slha(PMSSM_AL, "Advanced NN/AL/pmssm_al.in")    
    
    os.system('./SUSY/softsusy-4.1.9/softpoint.x leshouches < Advanced\ NN/AL/pmssm_al.in > Advanced\ NN/AL/pmssm_al.out')
            
          
    if os.path.getsize('/home/mathis/Stage M1/Advanced NN/AL/pmssm_al.out') == 0 :
        ratio[0]+=1
        return 0
        
    with open('Advanced NN/AL/pmssm_al.out') as slha_out:
        content = slha_out.read()
        if 'SOFTSUSY problem' in content:
            ratio[0]+=1
            return 0
        if 'LSP # Warning' in content:
            ratio[0]+=1
            return 0
    
    
    
    ratio[1]+=1
    commande = 'mv Advanced\ NN/AL/pmssm_al.out '
    nom_fichier = 'pmssm_al_succes_'+str(ratio[1])+'.out'
    os.system(commande+succes_path+nom_fichier)
    return 1  

In [5]:
os.system('rm -r Advanced\ NN/AL/softsusy_succes_IT/')
os.system('mkdir -p Advanced\ NN/AL/softsusy_succes_IT')
succes_path_IT = 'Advanced\ NN/AL/softsusy_succes_IT/'

ratio = [0,0]

In [6]:
#GeV [min,max]

M1_range = M2_range = M3_range = [50, 5000]
mA_range = [50, 5000]
tanB_range = [2, 60]
mu_range = [-10**4, 10**4]
At_range = Ab_range = Atau_range = [-10**4, 10**4]
Mq1L_range = Mq3L_range = [50, 5000]
MuR_range = MdR_range = MtR_range = MbR_range = [50, 5000]
MeL_range = MtauL_range = MeR_range = MtauR_range = [50, 5000]

PMSSM_range_big = [M1_range, M2_range, M3_range, mA_range, tanB_range, mu_range, At_range, Ab_range, 
              Atau_range, Mq1L_range, Mq3L_range, MuR_range, MdR_range, MtR_range, MbR_range,
             MeL_range, MtauL_range, MeR_range, MtauR_range]

PMSSM_min = np.array([i[0] for i in PMSSM_range_big])
PMSSM_max = np.array([i[1] for i in PMSSM_range_big])

PMSSM_sub = np.subtract(PMSSM_max, PMSSM_min)

PMSSM_range = [PMSSM_min,PMSSM_max,PMSSM_sub]

In [7]:
def write_slha(PMSSM_NN, nom_slha):
    f = open(nom_slha, "w")
    
    f.write("BLOCK MODSEL \n")
    f.write(" 1    0              #MSSM \n")
    
    f.write("BLOCK SMINPUTS \n")
    f.write("  1   1.279340000e+02     # alpha^(-1) SM MSbar(M1) \n")
    f.write("  2   1.663700000e-05     # G_Fermi \n")
    f.write("  3   0.1179              # alphas(MS) SM MSbar \n")
    f.write("  4   91.1876             # MZ(pole) \n")
    f.write("  5   4.18                # mb(mb) SM MSbar \n")
    f.write("  6   172.9               # mtop(pole) \n")
    f.write("  7   1.776860000e+00     #m mtau(pole) \n")
    
    f.write("BLOCK MINPAR \n")
    f.write("  3   "+str(PMSSM_NN[4])+"   #tanb \n")
    
    f.write("BLOCK VCKMIN   #CKM param (Wolfenstein) \n")
    f.write("  1   0.22650  #lambda \n")
    f.write("  2   0.790    #A \n")
    f.write("  3   0.141    #rho \n")
    f.write("  4   0.357    #eta \n")
    
    f.write("BLOCK EXTPAR \n")
    f.write("  0   "+str(-1.0)+"               # Q \n")
    f.write("  1   "+str(PMSSM_NN[0])+"        # M1 \n")
    f.write("  2   "+str(PMSSM_NN[1])+"        # M2 \n")
    f.write("  3   "+str(PMSSM_NN[2])+"        # M3 \n")
    f.write("  11   "+str(PMSSM_NN[6])+"       # At \n")
    f.write("  12   "+str(PMSSM_NN[7])+"       # Ab \n")
    f.write("  13   "+str(PMSSM_NN[8])+"       # Atau \n")
    f.write("  23   "+str(PMSSM_NN[5])+"       # Mu \n")
    f.write("  26   "+str(PMSSM_NN[3])+"       # Ma \n")
    f.write("  31   "+str(PMSSM_NN[15])+"      # MeL \n")
    f.write("  32   "+str(PMSSM_NN[15])+"      # MmuL \n")
    f.write("  33   "+str(PMSSM_NN[16])+"      # MstauL \n")
    f.write("  34   "+str(PMSSM_NN[17])+"      # MeR \n")
    f.write("  35   "+str(PMSSM_NN[17])+"      # MmuR \n")
    f.write("  36   "+str(PMSSM_NN[18])+"      # MstauR \n")
    f.write("  41   "+str(PMSSM_NN[9])+"       # Mq1L \n")
    f.write("  42   "+str(PMSSM_NN[9])+"       # Mq2L \n")
    f.write("  43   "+str(PMSSM_NN[10])+"      # Mq3L \n")
    f.write("  44   "+str(PMSSM_NN[11])+"      # MquR \n")
    f.write("  45   "+str(PMSSM_NN[11])+"      # MqcR \n")
    f.write("  46   "+str(PMSSM_NN[13])+"      # MqtR \n")
    f.write("  47   "+str(PMSSM_NN[12])+"      # MqdR \n")
    f.write("  48   "+str(PMSSM_NN[12])+"      # MqsR \n")
    f.write("  49   "+str(PMSSM_NN[14])+"      # MqbR \n")
    
    f.close()

In [ ]:
loss_evolution = []
epoch = 5
taille_batch = 1000

for k in range(epoch):
    print('Epochs {} / {}'.format(k+1,epoch))
    sum_loss = 0
    
    last = 0
    for i in tqdm(range(taille_batch,len(input_pred)+1,taille_batch)):
        batch = tf.convert_to_tensor(input_pred[last:i])
        last = i
        
        with tf.GradientTape() as tape:
            logits = model(batch)
            
            true_param = []
            for j in logits:
                rO = Oracle(j, PMSSM_range, ratio, succes_path_IT)
                if rO == 1:
                    idx = rd.randint(0,len(good_data))
                    true_param.append(good_data[idx])
                if rO == 0:
                    idx = rd.randint(0,len(bad_data))
                    true_param.append(bad_data[idx])
            true_param = tf.convert_to_tensor(true_param)
                    
            loss = loss_fn(true_param,logits)
            gradients = tape.gradient(loss, model.trainable_weights)

            sum_loss+=loss
    
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    loss_evolution.append(sum_loss.numpy()*taille_batch/len(input_pred))

Epochs 1 / 5


  0%|▏                                        | 2/500 [01:48<7:27:01, 53.86s/it]

In [ ]:
plt.plot([i for i in range(epoch)],loss_evolution,'ob')
plt.title('Évolution du loss')

print(loss_evolution)

In [ ]:
#test = tf.convert_to_tensor([0.6,0.7,0.8,0.9,1.0])
#output_test = model(test)
#print(output_test)

#for i in output_test:
#    print(Oracle(i, PMSSM_range, ratio, succes_path_IT))

In [ ]:
#test = tf.convert_to_tensor([0.0,0.1,0.2,0.3,0.4])
#output_test = model(test)
#print(output_test)

#for i in output_test:
#    print(Oracle(i, PMSSM_range, ratio, succes_path_IT))

In [ ]:
test = tf.convert_to_tensor([[rd.uniform(0,1) for j in range(19)] for i in range(5)])
output_test = model(test)
print(output_test)

for i in output_test:
    print(Oracle(i, PMSSM_range, ratio, succes_path_IT))

In [ ]:
print(model.trainable_weights)